<a href="https://colab.research.google.com/github/nagasatvika/semantic-similarity/blob/main/Word_similarity.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1. Importing pandas library
2. Using "as" keyword to create alias pd




In [1]:
import pandas as pd

1. Uploading and mounting the file into google drive  
2. Copy path and putting it in read_csv function of pandas for reading the file to DataFrame
3. Using a delimiter tab as the words are seperated by tab spaces

In [2]:
df = pd.read_csv("/content/drive/MyDrive/nlp/word.txt",delimiter="\t")

The DataFrame df contains 10 different columns as follows

In [3]:
df

,word1,word2,POS,SimLex999,conc(w1),conc(w2),concQ,Assoc(USF),SimAssoc333,SD(SimLex)
0,old,new,A,1.58,2.72,2.81,2,7.25,1,0.41
1,smart,intelligent,A,9.20,1.75,2.46,1,7.11,1,0.67
2,hard,difficult,A,8.77,3.76,2.21,2,5.94,1,1.19
3,happy,cheerful,A,9.55,2.56,2.34,1,5.85,1,2.18
4,hard,easy,A,0.95,3.76,2.07,2,5.82,1,0.93
...,...,...,...,...,...,...,...,...,...,...
994,join,acquire,V,2.85,2.86,2.93,2,0.00,0,0.99
995,send,attend,V,1.67,2.70,3.17,2,0.00,0,1.44
996,gather,attend,V,4.80,2.75,3.17,2,0.00,0,1.97
997,absorb,withdraw,V,2.97,3.11,3.04,2,0.00,0,1.75


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 999 entries, 0 to 998
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   word1        999 non-null    object 
 1   word2        999 non-null    object 
 2   POS          999 non-null    object 
 3   SimLex999    999 non-null    float64
 4   conc(w1)     999 non-null    float64
 5   conc(w2)     999 non-null    float64
 6   concQ        999 non-null    int64  
 7   Assoc(USF)   999 non-null    float64
 8   SimAssoc333  999 non-null    int64  
 9   SD(SimLex)   999 non-null    float64
dtypes: float64(5), int64(2), object(3)
memory usage: 78.2+ KB


#Constrained Setting

#Using SimLex999
Resources Referred
 [pandas](https://www.geeksforgeeks.org/different-ways-to-iterate-over-rows-in-pandas-dataframe/)

Simlex999 is a gold standard resource for evaluating distributional semantic models

In this code, used the SimLex999 column to find out the similarity based on following conditions

1. if 0 <= row['SimLex999'] < 3 then low similar
2. if 3 <= row['SimLex999'] < 6 then moderately similar
3. if 6 <= row['SimLex999'] < 9 then very similar
4. if 9 <= row['SimLex999'] <= 10 then highly similar



In [ ]:

def check_sim(row):
  if row['SimLex999']>=0 and row['SimLex999']<3:
    return "low similar"
  elif row['SimLex999']>=3 and row['SimLex999']<6:
    return "moderately similar"
  elif row['SimLex999']>=6 and row['SimLex999']<9:
    return "very similar"
  elif row['SimLex999']>=9 and row['SimLex999']<=10:
    return "highly similar"
df['similarity'] = df.apply(check_sim,axis=1)
for ind in df.index:
  print(f"{df['word1'][ind]}\t{df['word2'][ind]}\t{df['similarity'][ind]}")

old	new	low similar
smart	intelligent	highly similar
hard	difficult	very similar
happy	cheerful	highly similar
hard	easy	low similar
fast	rapid	very similar
happy	glad	highly similar
short	long	low similar
stupid	dumb	highly similar
weird	strange	very similar
wide	narrow	low similar
bad	awful	very similar
easy	difficult	low similar
bad	terrible	very similar
hard	simple	low similar
smart	dumb	low similar
insane	crazy	highly similar
happy	mad	low similar
large	huge	highly similar
hard	tough	very similar
new	fresh	very similar
sharp	dull	low similar
quick	rapid	highly similar
dumb	foolish	very similar
wonderful	terrific	very similar
strange	odd	highly similar
happy	angry	low similar
narrow	broad	low similar
simple	easy	highly similar
old	fresh	low similar
apparent	obvious	very similar
inexpensive	cheap	very similar
nice	generous	moderately similar
weird	normal	low similar
weird	odd	highly similar
bad	immoral	very similar
sad	funny	low similar
wonderful	great	very similar
guilty	ashamed	ve

PROS:        
1.Simlex999 is derived from the human judgements giving similarity score accuracy from moderate to highest

CONS:
1. It is only limited to only on one attribute i.e Simlex999



#Constrained Setting

#Wordnet using :
1. Wu palmer Similarity
2. path Similarity
1. Leacock Chordorow (LCH) Similarity







WORDNET:
Wordnet is large Lexical Database

Resource Referred: [link text](https://medium.com/@ompramod9921/wordnet-in-action-enhancing-nlp-capabilities-through-nltk-2aa922a57d43)()

1.   Importing NTLK Library
2.   downloading wordnet
3.   importing wordnet from nltk

1.  create alias "wn" using "as" keyword





In [4]:
import nltk
nltk.download('wordnet')
from nltk.corpus import wordnet as wn

[nltk_data] Downloading package wordnet to /root/nltk_data...




1. Extracting the first 20 rows from 'word1' and 'word2' of DataFrame df
1.   Define a function 'first_synset' which returns first synset of a given word(assuming that every word has atleast one synset) if that fails we return None
2.   Initialize an empty list named 'similarities' to store the result
1.   Using a for loop to iterate over the first 20 words then

 >Apply 'first_synset' function to the 'word1' and 'word2' to get their first synset

  >Calculating Wu-palmer similarity score between both synsets of 'word1' and 'word2'

 >Appending a tuple 'word1','word2' and 'similarity' to the list 'similarities'
1.   Creating a Dataframe of 'similarities' list using pandas library alias 'pd' named 'result'
2.   Print 'result' DataFrame










In [9]:
w1 = df['word1'][:999]
w2 = df['word2'][:999]
simlex = df['SimLex999'][:999]
def first_synset(word):
    synsets = wn.synsets(word)
    return synsets[0]
similarities = []
for word1, word2,score in zip(w1, w2,simlex):
    syn1 = first_synset(word1)
    syn2 = first_synset(word2)
    evaluation = score/10
    wup = syn1.wup_similarity(syn2)
    path = syn1.path_similarity(syn2)
    lch = syn1.lch_similarity(syn2) if syn1.pos()==syn2.pos() else 0
    similarities.append((word1,word2,wup,path,lch,evaluation))
results = pd.DataFrame(similarities,columns=['word1','word2','wup','path','lch','evaluation'])
print(results)

      word1        word2       wup      path       lch  evaluation
0       old          new  0.222222  0.125000  0.000000       0.158
1     smart  intelligent  0.166667  0.090909  0.000000       0.920
2      hard    difficult  1.000000  1.000000  0.693147       0.877
3     happy     cheerful  0.500000  0.333333 -0.405465       0.955
4      hard         easy  0.500000  0.333333 -0.405465       0.095
..      ...          ...       ...       ...       ...         ...
994    join      acquire  0.222222  0.125000  0.000000       0.285
995    send       attend  0.333333  0.200000  1.648659       0.167
996  gather       attend  0.153846  0.083333  0.000000       0.480
997  absorb     withdraw  0.250000  0.142857  1.312186       0.297
998  attend       arrive  0.400000  0.250000  1.871802       0.608

[999 rows x 6 columns]


In [10]:
from scipy.stats import pearsonr
pearson_wup = pearsonr(results['wup'],results['evaluation'])
pearson_path = pearsonr(results['path'],results['evaluation'])
pearson_lch = pearsonr(results['lch'],results['evaluation'])
print ("Pearson coffecient correlation for wup and evaluation:",pearson_wup)
print ("Pearson coffecient correlation for path and evaluation:",pearson_path)
print ("Pearson coffecient correlation for lch and evaluation:",pearson_lch)

Pearson coffecient correlation for wup and evaluation: PearsonRResult(statistic=0.28425968333800644, pvalue=5.046714001105548e-20)
Pearson coffecient correlation for path and evaluation: PearsonRResult(statistic=0.38518051326537983, pvalue=1.1116710392775524e-36)
Pearson coffecient correlation for lch and evaluation: PearsonRResult(statistic=0.2272620190169458, pvalue=3.611045734846462e-13)


#Unconstrained Setting

#**Using Word2vec Similarity**





[resource followed for the below task](https://tedboy.github.io/nlps/generated/generated/gensim.models.Word2Vec.similarity.html)

In [13]:
import gensim.downloader as api
import pandas as pd


In [14]:
model = api.load('word2vec-google-news-300')


[==================================================] 100.0% 1662.8/1662.8MB downloaded


In [20]:
w1 = df['word1'][:999]
w2 = df['word2'][:999]
simlex = df['SimLex999'][:999]

In [21]:
similarities = []
for word1, word2,score in zip(w1, w2,simlex):
  similarity = model.similarity(word1, word2)
  evaluation_parameter = score/10
  res = (word1, word2, similarity,evaluation_parameter)
  similarities.append(res)
results_word2vec = pd.DataFrame(similarities, columns=['word1', 'word2', 'word2vec_similarity','evaluation_parameter'])
print(results_word2vec)


      word1        word2  word2vec_similarity  evaluation_parameter
0       old          new             0.222780                 0.158
1     smart  intelligent             0.649528                 0.920
2      hard    difficult             0.602575                 0.877
3     happy     cheerful             0.383774                 0.955
4      hard         easy             0.470963                 0.095
..      ...          ...                  ...                   ...
994    join      acquire             0.262319                 0.285
995    send       attend             0.313500                 0.167
996  gather       attend             0.332902                 0.480
997  absorb     withdraw             0.216904                 0.297
998  attend       arrive             0.365385                 0.608

[999 rows x 4 columns]


In [25]:
from scipy.stats import pearsonr
pearson_w2v = pearsonr(results_word2vec['word2vec_similarity'],results_word2vec['evaluation_parameter'])
print ("Pearson coffecient correlation for peason and evaluation:",pearson_w2v)

Pearson coffecient correlation for peason and evaluation: PearsonRResult(statistic=0.45392820864420297, pvalue=6.17306308815303e-52)


#Spacy Similarity method
Resources referred :

1.  [tutorial](https://campus.datacamp.com/courses/advanced-nlp-with-spacy/large-scale-data-analysis-with-spacy?ex=8)
2.  [paper](https://www.irjmets.com/uploadedfiles/paper/issue_9_september_2022/30252/final/fin_irjmets1664350077.pdf)





Using pip to install spacy library





In [26]:
!pip install spacy




1.   Using 'python -m spacy' which means to run spacy module
2.   Downloading en_core_web_md which is Medium-sized English model



In [27]:
!python -m spacy download en_core_web_md

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 MB 17.3 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_md')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


1.   Importing spacy library
2.   loading en_core_web_md i.e medium sized english pre-trained model



In [28]:
import spacy
nlp = spacy.load('en_core_web_md')

1. Extracting the first 10 rows from 'word1' and 'word2' of DataFrame df
2. Initialize an empty list named 'similarities' to store the result
1. Using a for loop to iterate over the first 20 words then
> applying nlp for word1 and word2 which creates a spacy document

 >  we use similarity method to find the similarity scores of word1 and word2

 > Appending a tuple ('word1','word2','similarity') to the list 'similarities'



4.   Creating a Dataframe of 'similarities' list using pandas library alias 'pd' named 'result'
2.   Print 'result' DataFrame



In [31]:
w1 = df['word1'][:999]
w2 = df['word2'][:999]
simlex = df['SimLex999'][:999]
similarities = []

for word1,word2,score in zip(w1,w2,simlex):
  W1 = nlp(word1)
  W2 = nlp(word2)
  Similarity = W1.similarity(W2)
  siml = score/10
  res = (word1,word2,Similarity,siml)
  similarities.append(res)
result = pd.DataFrame(similarities,columns=['word1','word2','Similarity','simlex999'])
print(result)

      word1        word2  Similarity  simlex999
0       old          new    0.157152      0.158
1     smart  intelligent    0.592976      0.920
2      hard    difficult    0.698994      0.877
3     happy     cheerful    0.606921      0.955
4      hard         easy    0.554700      0.095
..      ...          ...         ...        ...
994    join      acquire    0.320770      0.285
995    send       attend    0.379135      0.167
996  gather       attend    0.478372      0.480
997  absorb     withdraw    0.349713      0.297
998  attend       arrive    0.421151      0.608

[999 rows x 4 columns]


In [33]:
from scipy.stats import pearsonr
pearson_spacy= pearsonr(result['Similarity'],result['simlex999'])
print ("Pearson coffecient correlation for similarity scores  and evaluation:",pearson_spacy)

Pearson coffecient correlation for similarity scores  and evaluation: PearsonRResult(statistic=0.43324437835288276, pvalue=5.7194843423130715e-47)


#Using Sentence Transformers
Resource Referred :

1.  [sbert](https://www.sbert.net/docs/sentence_transformer/usage/semantic_textual_similarity.html)







1.  Installing **sentence-transformers**
2.  **-q** is a flag used to supress the output during installation process



In [34]:
%pip install sentence-transformers -q


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 36.6 MB/s eta 0:00:00


1. Importing **SentenceTransformer** CLASS from **sentence_transformers**
2. util is a module which includes functions such as cosine similarity
1.  The **SentenceTransformer** Class is *instantiated* with **all-MiniLM-L6-v2**
 which generates the *vector* *representations* of given *sentence*





In [35]:
from sentence_transformers import SentenceTransformer, util
model = SentenceTransformer('all-MiniLM-L6-v2')

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]



1.  Extracting the first 20 rows from 'word1' and 'word2' of DataFrame df
2.  Generating vector representations/embeddings of words w1,w2 using SentenceTransformer model
1. Then Calculating the cosine similarity between the vector representations/embeddings
2. Using a for for iterating through the first 20 words and printing similarity scores





In [39]:
w1 = df['word1'][:999]
w2 = df['word2'][:999]
simlex = df['SimLex999'][:999]
cosine = []
eval = []
embeddings1 = model.encode(w1, convert_to_tensor=True)
embeddings2 = model.encode(w2, convert_to_tensor=True)
cosine_score = util.cos_sim(embeddings1,embeddings2)
for i in range(999):
  cosine_score_value = cosine_score[i][i]
  cosine.append(cosine_score_value)
  sim = simlex[i]/10
  eval.append(sim)
  print("{} \t\t {} \t\t  {:.4f} \t  {:.4f}".format(w1[i], w2[i], cosine_score_value, sim))


old 		 new 		  0.5034 	  0.1580
smart 		 intelligent 		  0.7494 	  0.9200
hard 		 difficult 		  0.8246 	  0.8770
happy 		 cheerful 		  0.5541 	  0.9550
hard 		 easy 		  0.7239 	  0.0950
fast 		 rapid 		  0.7319 	  0.8750
happy 		 glad 		  0.5697 	  0.9170
short 		 long 		  0.6555 	  0.1230
stupid 		 dumb 		  0.7669 	  0.9580
weird 		 strange 		  0.8870 	  0.8930
wide 		 narrow 		  0.8235 	  0.1030
bad 		 awful 		  0.6512 	  0.8420
easy 		 difficult 		  0.7745 	  0.0580
bad 		 terrible 		  0.7928 	  0.7780
hard 		 simple 		  0.5171 	  0.1380
smart 		 dumb 		  0.6009 	  0.0550
insane 		 crazy 		  0.8316 	  0.9570
happy 		 mad 		  0.3313 	  0.0950
large 		 huge 		  0.8102 	  0.9470
hard 		 tough 		  0.7784 	  0.8050
new 		 fresh 		  0.5359 	  0.6830
sharp 		 dull 		  0.5933 	  0.0600
quick 		 rapid 		  0.5722 	  0.9700
dumb 		 foolish 		  0.5438 	  0.6670
wonderful 		 terrific 		  0.6722 	  0.8630
strange 		 odd 		  0.8245 	  0.9020
happy 		 angry 		  0.3906 	  0.1280
narrow 		 broad 		  

In [40]:
from scipy.stats import pearsonr
pearson_st = pearsonr(cosine,eval)
print ("Pearson coffecient correlation for similarity scores  and evaluation:",pearson_st)

Pearson coffecient correlation for similarity scores  and evaluation: PearsonRResult(statistic=0.4599460243019872, pvalue=1.9054343611459298e-53)
